### Поиск организаций, имеющих лицензии

In [1]:
%%time

import pandas as pd
import numpy as np
import os
import sys
from datetime import date

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")
# пути и списки файлов
dir_path = "C:/work/licenses"
# if len(os.listdir(dir_path)) > 4:
#     print(
#         "В целевой папке должны быть только актуальный Реестр МО, файл лицензий и 'Силовики'"
#     )
#     sys.exit()
for file_name in os.listdir(dir_path):
    if file_name.split("_")[0] == "1.2.643.5.1.13.13.11.1461":
        ref_path = dir_path + "/" + file_name
    elif file_name.split(" ")[0] == "Силовики" or file_name.split(" ")[0] == "силовики":
        army_path = dir_path + "/" + file_name
    else:
        lic_path = dir_path + "/" + file_name

df_ref = pd.read_excel(ref_path, sheet_name="Справочник", index_col=None, dtype="str")
df_lic = pd.read_excel(lic_path, sheet_name="Лист 1", index_col=None, dtype="str")
df_lic = df_lic.drop(
    index=range(0, df_lic[df_lic["Unnamed: 1"] == "Дата"].index[0])
).reset_index(drop=True)
df_lic.columns = df_lic.iloc[0]
df_lic = df_lic.drop(index=0).reset_index(drop=True)
df_army = pd.read_excel(
    army_path, sheet_name="Лицензии+ФРМО", index_col=None, dtype="str"
)
df_army_s = df_army[["№", "Дополнительный признак для фильтрации"]].rename(columns = {"Дополнительный признак для фильтрации": "special_info"})
df_lic_s = df_lic[["№", "ИНН", "ОГРН"]]
med_name_list = []
for name in df_ref["medicalSubjectName"].unique():
    if "едицинская" in name:
        med_name_list.append(name)
med_name_list
df_ref_s = df_ref[
    (df_ref["medicalSubjectName"].isin(med_name_list)) & (df_ref["deleteDate"].isnull())
][
    [
        "oid",
        "nameFull",
        "nameShort",
        "inn",
        "ogrn",
        "regionName",
        "moDeptName",
        "parentId",
    ]
]
df_lic_inn = df_lic_s.merge(df_ref_s, left_on="ИНН", right_on="inn", how="left")
df_lic_inn["origin"] = "только по ИНН"
df_lic_ogrn = df_lic_s.merge(df_ref_s, left_on="ОГРН", right_on="ogrn", how="left")
df_lic_ogrn["origin"] = "только по ОГРН"
df_lic_result = (
    pd.concat([df_lic_inn, df_lic_ogrn]).sort_values(by="№").reset_index(drop=True)
)
for index, row in df_lic_result.iterrows():
    if pd.isnull(row["oid"]):
        df_lic_result.loc[index, "origin"] = ""
for lic_num in df_lic_result["№"].unique():
    df_lic_0 = df_lic_result[df_lic_result["№"] == lic_num]
    for oid_num in df_lic_0["oid"].unique():
        df_lic_1 = df_lic_0[df_lic_0["oid"] == oid_num]
        if df_lic_1.shape[0] == 2:
            df_lic_result.loc[
                df_lic_1.index[0] : df_lic_1.index[-1], "origin"
            ] = "и по ИНН, и по ОГРН"
df_lic_result.drop_duplicates(inplace=True)
df_lic_result_a = df_lic_result.copy()
df_lic_result = df_lic_result_a.merge(df_army_s, left_on="№", right_on="№", how="left")
# for index, row in df_lic_result.iterrows():
#     if pd.isnull(row["oid"]):
#         df_lic_result.loc[index, "origin"] = ""
df_lic_result.drop(["inn", "ogrn"], axis=1, inplace=True)
today = date.today().strftime("%Y-%m-%d")
df_lic_result.to_csv(
    dir_path + "/" + "licenses_report_" + today + ".csv",
    sep=";",
    encoding="utf-8-sig",
    index=False,
)
path_save = dir_path + "/" + "licenses_report_" + today + ".xlsx"
writer = pd.ExcelWriter(path_save, engine="xlsxwriter")
df_lic_result.to_excel(writer, sheet_name="лицензии+НСИ", index=False)
writer.close()
df_lic_result.head(5)

CPU times: total: 11.6 s
Wall time: 47.8 s


,№,ИНН,ОГРН,oid,nameFull,nameShort,regionName,moDeptName,parentId,origin,special_info
0,Л041-00110-01/00357877,0105025965,1020100703842,1.2.643.5.1.13.13.12.2.1.32,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,ГБУЗРА АРКБ,Республика Адыгея (Адыгея),Органы исполнительной власти субъектов Российс...,NaN,"и по ИНН, и по ОГРН",NaN
1,Л041-00110-01/00361707,0105014177,1020100698595,1.2.643.5.1.13.13.12.3.1.3434,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВА...,"ФГБОУ ВО ""МГТУ"", МГТУ, МАЙКОПСКИЙ ГОСУДАРСТВЕН...",Республика Адыгея (Адыгея),Министерство образования и науки Российской Фе...,NaN,"и по ИНН, и по ОГРН",NaN
2,Л041-00110-01/00571612,0105047430,1060105011361,1.2.643.5.1.13.13.12.2.1.15354,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,"ФКУЗ ""МСЧ МВД РОССИИ ПО РЕСПУБЛИКЕ АДЫГЕЯ""",Республика Адыгея,Министерство внутренних дел Российской Федерации,NaN,"и по ИНН, и по ОГРН",МСЧ / поликлиника / госпиталь МВД
3,Л041-00110-02/00323564,0275006624,1020202776109,1.2.643.5.1.13.13.12.2.2.17875,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,"ФКУЗ ""МСЧ МВД РОССИИ ПО РЕСПУБЛИКЕ БАШКОРТОСТАН""",Республика Башкортостан,Министерство внутренних дел Российской Федерации,NaN,"и по ИНН, и по ОГРН",МСЧ / поликлиника / госпиталь МВД
4,Л041-00110-02/00340505,0274064870,1030204207582,1.2.643.5.1.13.13.12.2.2.11156,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ НАУЧНОЕ ...,УФИЦ РАН,Республика Башкортостан,Министерство образования и науки Российской Фе...,NaN,"и по ИНН, и по ОГРН",NaN


In [62]:
%%time

import pandas as pd
import numpy as np
import os
import sys
from datetime import date

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")
# пути и списки файлов
dir_path = "C:/work/licenses"
# if len(os.listdir(dir_path)) > 4:
#     print(
#         "В целевой папке должны быть только актуальный Реестр МО, файл лицензий и 'Силовики'"
#     )
#     sys.exit()
for file_name in os.listdir(dir_path):
    if file_name.split("_")[0] == "1.2.643.5.1.13.13.11.1461":
        ref_path = dir_path + "/" + file_name
    elif file_name.split(" ")[0].split(".")[0].lower() == "силовики":
        army_path = dir_path + "/" + file_name
    elif file_name.split("_")[0].split(".")[0].lower() == "лицензии":
        lic_path = dir_path + "/" + file_name

df_ref = pd.read_excel(ref_path, sheet_name="Справочник", index_col=None, dtype="str")
df_lic = pd.read_excel(lic_path, sheet_name="Лист 1", index_col=None, dtype="str")
df_lic = df_lic.drop(
    index=range(0, df_lic[df_lic["Unnamed: 1"] == "Дата"].index[0])
).reset_index(drop=True)
df_lic.columns = df_lic.iloc[0]
df_lic = df_lic.drop(index=0).reset_index(drop=True)
df_army = pd.read_excel(
    army_path, sheet_name="Лицензии+ФРМО", index_col=None, dtype="str"
)
df_army_s = df_army[["№", "Дополнительный признак для фильтрации"]].rename(columns = {"Дополнительный признак для фильтрации": "special_info"})
df_lic_s = df_lic[["№", "ИНН", "ОГРН"]]
med_name_list = []
for name in df_ref["medicalSubjectName"].unique():
    if "едицинская" in name:
        med_name_list.append(name)
med_name_list
df_ref_s = df_ref[
    (df_ref["medicalSubjectName"].isin(med_name_list)) & (df_ref["deleteDate"].isnull())
][
    [
        "oid",
        "nameFull",
        "nameShort",
        "inn",
        "ogrn",
        "regionName",
        "moDeptName",
        "parentId",
    ]
]
df_lic_inn = df_lic_s.merge(df_ref_s, left_on="ИНН", right_on="inn", how="left")
df_lic_inn["origin"] = "ИНН"
df_lic_ogrn = df_lic_s.merge(df_ref_s, left_on="ОГРН", right_on="ogrn", how="left")
df_lic_ogrn["origin"] = "ОГРН"
df_lic_result = (
    pd.concat([df_lic_inn, df_lic_ogrn]).sort_values(by="№").reset_index(drop=True)
)
df_lic_result["oid"] = df_lic_result["oid"].fillna("не найден")
for index, row in df_lic_result.iterrows():
    if pd.isnull(row["oid"]):
        df_lic_result.loc[index, "origin"] = ""
for lic_num in df_lic_result["№"].unique():
    df_lic_0 = df_lic_result[df_lic_result["№"] == lic_num]
    for oid_num in df_lic_0["oid"].unique():
        df_lic_1 = df_lic_0[df_lic_0["oid"] == oid_num]
        if df_lic_1.shape[0] == 2:
            df_lic_result.loc[
                df_lic_1.index[0] : df_lic_1.index[-1], "origin"
            ] = "ИНН и ОГРН"
df_lic_result.drop_duplicates(inplace=True)
df_lic_result_a = df_lic_result.copy()
df_lic_result = df_lic_result_a.merge(df_army_s, left_on="№", right_on="№", how="left")
df_lic_result.drop(["inn", "ogrn"], axis=1, inplace=True)
today = date.today().strftime("%Y-%m-%d")

lic_inn_dict = {}
for lic_num in df_lic_inn["№"].unique():
    df_lic_l = df_lic_inn[df_lic_inn["№"] == lic_num]
    lic_inn_dict[lic_num] = df_lic_l["oid"].tolist()
df_inn = pd.DataFrame(columns=["№", "найдено по ИНН"])
for key, value in lic_inn_dict.items():
    df_inn.loc[len(df_inn.index)] = [key, value]

lic_ogrn_dict = {}
for lic_num in df_lic_ogrn["№"].unique():
    df_lic_l = df_lic_ogrn[df_lic_ogrn["№"] == lic_num]
    lic_ogrn_dict[lic_num] = df_lic_l["oid"].tolist()
df_ogrn = pd.DataFrame(columns=["№", "найдено по ОГРН"])
for key, value in lic_ogrn_dict.items():
    df_ogrn.loc[len(df_ogrn.index)] = [key, value]

df_lic_list = df_inn.merge(df_ogrn, left_on="№", right_on="№")

path_save = dir_path + "/" + "licenses_report_" + today + ".xlsx"
writer = pd.ExcelWriter(path_save, engine="xlsxwriter")
df_lic_result.to_excel(writer, sheet_name="лицензии+НСИ", index=False)
df_lic_list.to_excel(writer, sheet_name="oid списком", index=False)
writer.close()
df_lic_result.head(5)

CPU times: total: 22.6 s
Wall time: 1min 16s


,№,ИНН,ОГРН,oid,nameFull,nameShort,regionName,moDeptName,parentId,origin,special_info
0,Л041-00110-01/00357877,0105025965,1020100703842,1.2.643.5.1.13.13.12.2.1.32,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,ГБУЗРА АРКБ,Республика Адыгея (Адыгея),Органы исполнительной власти субъектов Российс...,NaN,ИНН и ОГРН,NaN
1,Л041-00110-01/00361707,0105014177,1020100698595,1.2.643.5.1.13.13.12.3.1.3434,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВА...,"ФГБОУ ВО ""МГТУ"", МГТУ, МАЙКОПСКИЙ ГОСУДАРСТВЕН...",Республика Адыгея (Адыгея),Министерство образования и науки Российской Фе...,NaN,ИНН и ОГРН,NaN
2,Л041-00110-01/00571612,0105047430,1060105011361,1.2.643.5.1.13.13.12.2.1.15354,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,"ФКУЗ ""МСЧ МВД РОССИИ ПО РЕСПУБЛИКЕ АДЫГЕЯ""",Республика Адыгея,Министерство внутренних дел Российской Федерации,NaN,ИНН и ОГРН,МСЧ / поликлиника / госпиталь МВД
3,Л041-00110-02/00323564,0275006624,1020202776109,1.2.643.5.1.13.13.12.2.2.17875,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,"ФКУЗ ""МСЧ МВД РОССИИ ПО РЕСПУБЛИКЕ БАШКОРТОСТАН""",Республика Башкортостан,Министерство внутренних дел Российской Федерации,NaN,ИНН и ОГРН,МСЧ / поликлиника / госпиталь МВД
4,Л041-00110-02/00340505,0274064870,1030204207582,1.2.643.5.1.13.13.12.2.2.11156,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ НАУЧНОЕ ...,УФИЦ РАН,Республика Башкортостан,Министерство образования и науки Российской Фе...,NaN,ИНН и ОГРН,NaN


Л041-00110-38/00368032
Л041-01162-50/00368299

In [63]:
df_lic_result[df_lic_result["№"] == "Л041-00110-38/00368032"]

,№,ИНН,ОГРН,oid,nameFull,nameShort,regionName,moDeptName,parentId,origin,special_info
511,Л041-00110-38/00368032,3828006039,1023802527627,1.2.643.5.1.13.13.12.2.38.16505,Филиал «Медицинская часть № 7» федерального ка...,Филиал «Медицинская часть № 7» ФКУЗ МСЧ-38 ФСИ...,Иркутская область,Федеральная служба исполнения наказаний,1.2.643.5.1.13.13.12.2.38.9334,ИНН,NaN
512,Л041-00110-38/00368032,3828006039,1023802527627,1.2.643.5.1.13.13.12.2.38.16506,Филиал «Медицинская часть № 8» федерального ка...,Филиал «Медицинская часть № 8» ФКУЗ МСЧ-38 ФСИ...,Иркутская область,Федеральная служба исполнения наказаний,1.2.643.5.1.13.13.12.2.38.9334,ИНН,NaN
513,Л041-00110-38/00368032,3828006039,1023802527627,1.2.643.5.1.13.13.12.2.38.16500,Филиал «Медицинская часть № 1» федерального ка...,Филиал «Медицинская часть № 1» ФКУЗ МСЧ-38 ФСИ...,Иркутская область,Федеральная служба исполнения наказаний,1.2.643.5.1.13.13.12.2.38.9334,ИНН,NaN
514,Л041-00110-38/00368032,3828006039,1023802527627,1.2.643.5.1.13.13.12.2.38.16509,Филиал «Медицинская часть № 11» федерального к...,Филиал «Медицинская часть № 11» ФКУЗ МСЧ-38 ФС...,Иркутская область,Федеральная служба исполнения наказаний,1.2.643.5.1.13.13.12.2.38.9334,ИНН,NaN
515,Л041-00110-38/00368032,3828006039,1023802527627,1.2.643.5.1.13.13.12.2.38.16507,Филиал «Медицинская часть № 9» федерального ка...,Филиал «Медицинская часть № 9» ФКУЗ МСЧ-38 ФСИ...,Иркутская область,Федеральная служба исполнения наказаний,1.2.643.5.1.13.13.12.2.38.9334,ИНН,NaN
516,Л041-00110-38/00368032,3828006039,1023802527627,1.2.643.5.1.13.13.12.2.38.16511,Филиал «Медицинская часть № 13» федерального к...,Филиал «Медицинская часть № 13» ФКУЗ МСЧ-38 ФС...,Иркутская область,Федеральная служба исполнения наказаний,1.2.643.5.1.13.13.12.2.38.9334,ИНН,NaN
517,Л041-00110-38/00368032,3828006039,1023802527627,1.2.643.5.1.13.13.12.2.38.16498,Филиал «Центр государственного санитарно-эпиде...,Филиал « ЦГСН» ФКУЗ МСЧ-38 ФСИН России,Иркутская область,Федеральная служба исполнения наказаний,1.2.643.5.1.13.13.12.2.38.9334,ИНН,NaN
518,Л041-00110-38/00368032,3828006039,1023802527627,1.2.643.5.1.13.13.12.2.38.16501,Филиал «Медицинская часть № 2» федерального ка...,Филиал «Медицинская часть № 2» ФКУЗ МСЧ-38 ФСИ...,Иркутская область,Федеральная служба исполнения наказаний,1.2.643.5.1.13.13.12.2.38.9334,ИНН,NaN
519,Л041-00110-38/00368032,3828006039,1023802527627,1.2.643.5.1.13.13.12.2.38.16514,Филиал «Больница № 2» федерального казенного у...,Филиал «Больница № 2» ФКУЗ МСЧ-38 ФСИН России,Иркутская область,Федеральная служба исполнения наказаний,1.2.643.5.1.13.13.12.2.38.9334,ИНН,NaN
520,Л041-00110-38/00368032,3828006039,1023802527627,1.2.643.5.1.13.13.12.2.38.16512,Филиал «Медицинская часть № 14» федерального к...,Филиал «Медицинская часть № 14» ФКУЗ МСЧ-38 ФС...,Иркутская область,Федеральная служба исполнения наказаний,1.2.643.5.1.13.13.12.2.38.9334,ИНН,NaN
